# PSB sequence

This includes the correct ramp rate and SET feedback as well.

In [ ]:
%matplotlib inline
import local_broom

In [ ]:
from zhinst.toolkit import Session, CommandTable, Sequence, Waveforms, SHFQAChannelMode

import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

import time
from monty import Monty

# Connect

In [ ]:
# connect to instruments

DEVICE_ID = 'DEV12158'
SERVER_HOST = 'localhost'

# connect to data server
session = Session(SERVER_HOST)

# connect to device
device = session.connect_device(DEVICE_ID)

In [ ]:
#session.disconnect_device(DEVICE_ID)

In [ ]:
experiment = {
    "desc": "PSB sequence."
}

monty = Monty("rf.psb", experiment)

In [2]:

# lengths and averages
averages = 1  # python averages
seq_averages = 10000  # seqc averages

detuning_pts = 100
j_pts = 100

# readout signals
readout_gain = 0.95  # If we set this to 1, then output overloads
readout_freq = 406.2e6  # (Hz)

# drive line amplitudes (maximum 340mV, give in terms of V)
init_amps = [0.05, 0, -0.05]  # [p1, j, p2]
read_amps = [0.02, 0, -0.02]  # [p1, j, p2]

detuning_list = np.linspace(0, 0.1, detuning_pts)  # for p1, take the negative of this list
j_list = np.linspace(-0.3, 0, j_pts)

# timings in seconds
init_len = 600e-6
read_len = 100e-6
wait_and_settle = 3e-3
trigger_time = 6e-3  # internal trigger holdoff time (metronome)
buffer = 32  # in terms of samples

# AC decay ramp rate
ramp_rate = 13e-5  # V/sample

# powers
input_pwr = -0
output_pwr = -0
dr_pwr = 0

# gate sampling time
samplingDivider = 6

# internal
read_samples = timeToSamples(read_len, samplingDivider)


NameError: name 'np' is not defined

In [ ]:
# check we can fit all the points into memory
MAX_MEASUREMENTS = 2**19
if seq_averages*detuning_pts*j_pts > MAX_MEASUREMENTS:
    raise OverflowError("Requested too many points to be measured. (Use around 500,000 points)")

In [ ]:
params = {
    "samplingDivider": samplingDivider,
    
    "averages": averages,
    "seq_averages": seq_averages,
    "detuning_pts": detuning_pts,
    "j_pts": j_pts,
    
    "readout_freq": readout_freq,
    "readout_gain": readout_gain,
    
    "init_amps": init_amps,
    "read_amps": read_amps,
    "detuning_list": detuning_list,
    "j_list": j_list,

    "ramp_rate": ramp_rate,
    
    "input_pwr": input_pwr,
    "output_pwr": output_pwr,
    "dr_pwr": dr_pwr,

    "init_len": init_len,
    "read_len": read_len,
    "wait_and_settle": wait_and_settle,
    "trigger_time": trigger_time,

}

# Functions

In [ ]:
def timeToSamples(time, samplingRateDivider):
    # returns the number of samples divisible by 16 given a time (in seconds) and sampling rate divider
    samples_raw = time * (1/(2**samplingRateDivider))/0.5e-9
    samples_modulo = int(samples_raw) % 16
    samples = int(samples_raw) - int(samples_modulo)
    return samples

# custom curve fit
amps = [1, 0.85, 0.75, 0.65,  0.5, 0.35, 0.25, 0.2, 0.125, 0.05]
voltages = [0.34, 0.268, 0.207, 0.158, 0.094, 0.0481,  0.026, 0.0183, 0.01, 0.0057]

alt_fit = np.poly1d(np.polyfit(amps, voltages, 2))
# Hard coding alt_fit values in case something goes horribly wrong

def voltToDbm(volt, dbmrange):
    # Ok yes this can be better, deal with it
    if dbmrange != 0:
        raise Exception("This function only works with a dBm range of 0.")
    
    if volt > 0.34 or volt < -0.34:
        raise ValueError(f"Given voltage ({volt} V) is greater than max output of SHFQC (0.34 V)")
    
    if volt < 0:
        amplitude = 1/300*(np.sqrt(3e5*-volt + 529) - 23)
        return -amplitude
    else:
        amplitude = 1/300*(np.sqrt(3e5*volt + 529) - 23)
        return amplitude


def get_results(result_node, timeout):
    wave_data_captured = {}
    wave_data_captured[result_node] = False
    start_time = time.time()
    captured_data = {}
    while not all(wave_data_captured.values()):
        if start_time + timeout < time.time():
            print(captured_data)
            raise TimeoutError('Timeout before all samples collected.')
        test = session.poll()
        for node, value in test.items():
            node = session.raw_path_to_node(node)
            for v in value:
                if node not in captured_data:
                    captured_data[node] = [v['vector']]
                else:
                    captured_data[node].append(v['vector'])
            if len(captured_data[node]) >= 1:  # readout 1 point
                wave_data_captured[node] = True
                # total_num_data = sum([len(element) for element in captured_data[node]])
    data = captured_data[result_node][0]
    return data

def cmdtable(ct, amplitude, length, wave_index, ct_index):
    """
    Load a default command table with a sin/cos wave (used throughout the documentation)
    """
    ct.table[ct_index].waveform.index = wave_index
    ct.table[ct_index].amplitude00.value = amplitude  # all in dBm
    ct.table[ct_index].amplitude01.value = -amplitude
    ct.table[ct_index].amplitude10.value = amplitude
    ct.table[ct_index].amplitude11.value = amplitude
    ct.table[ct_index].waveform.length = length  # in samples
    ct.table[ct_index].waveform.samplingRateDivider = samplingDivider  # inherit global

# Channels

In [ ]:
# Create channel maps for simplicity

chan = {
    "measure": device.qachannels[0],  # measure and acquire lines
    "ST": device.sgchannels[0],
    "P1": device.sgchannels[1],  # drive P1 line
    "P2": device.sgchannels[2],  # drive P2 line
    "J": device.sgchannels[3],  # drive J line
}

drive_chans = ["ST", "P1", "P2", "J"]  # match keys above

In [ ]:
with device.set_transaction():
    # setup drive channels
    for c in drive_chans:
        chan[c].output.range(dr_pwr)  # in dBm
        chan[c].output.rflfpath(0)  # use LF not RF (1 for RF)
    
        # set the center synth frequency (oscillator frequency)
        synth = chan[c].synthesizer()
        device.synthesizers[synth].centerfreq(0)  # in Hz
        chan[c].output.on(1)  # enable output
    
        chan[c].awg.outputamplitude(1.0)  # overall amplitude scaling factor (don't really need to change)
        chan[c].oscs[0].freq(0)  # oscillator 1 frequency (Hz) disable for DC
        chan[c].oscs[1].freq(0)  # oscillator 2 frequency (Hz)
        chan[c].awg.modulation.enable(1)  # start digital modulation
    
        chan[c].marker.source(0)  # setup the AWG trigger 1 (is this an input trigger option? doesn't seem necessary)
        # see manual page p235 for all trigger options
        chan[c].awg.auxtriggers[0].channel(8)  # 8=use internal trigger, 1024=use software trigger

    # setup measure channel
    
    chan["measure"].output.rflfpath(0)  # use LF mode not RF (for signals under 600Mhz)
    chan["measure"].input.rflfpath(0)
    chan["measure"].oscs[0].freq(readout_freq)  # CW frequency (in LF mode)
    chan["measure"].oscs[0].gain(readout_gain)  # If we set this to 1, then output overloads

    # configure these based on how the sweeper works internally
    # See https://docs.zhinst.com/zhinst-utils/en/latest/_modules/zhinst/utils/shf_sweeper.html#ShfSweeper
    chan["measure"].spectroscopy.delay(0)  # integration delay in units of second
    chan["measure"].spectroscopy.length(timeToSamples(read_time, samplingDivider))  # integration time length in units of number of samples (usually integration_time*sampling_rate)
    # setup when the spectroscopy is triggered
    chan["measure"].spectroscopy.trigger.channel("chan0seqtrig0")  # make sure to use the trigger coming from the sequencer code
    # setup result parameters
    chan["measure"].spectroscopy.result.averages(1)  # number of averages (always average in software not hardware)
    chan["measure"].spectroscopy.result.length(seq_averages*(read_lens-offset))  # number of results
    chan["measure"].spectroscopy.result.enable(0)  # disable result logger

    chan["measure"].configure_channel(
        center_frequency=0,  # in units of Hz  # minimum of 600MHz for RF mode
        input_range=input_pwr,  # in units of dBm
        output_range=output_pwr,  # in units of dBm
        mode=SHFQAChannelMode.SPECTROSCOPY,  # SHFQAChannelMode.READOUT or SHFQAChannelMode.SPECTROSCOPY
    )
    
    chan["measure"].input.on(1)
    chan["measure"].output.on(1)

    chan["measure"].generator.auxtriggers[1].channel("inttrig")  # i believe this is overwritten by the following line
    chan["measure"].generator.configure_sequencer_triggering(
        aux_trigger=8,  # alternatively use 8=internal trigger, or "software_trigger0" to use the software triggering system
        play_pulse_delay=0
    )


# Ramped pulses

In [5]:
# # define waveforms as list of real-values arrays

# waves_p1 = []
# waves_p2 = []
# for i, val in enumerate(detuning_list):

#     wv_p1 = np.linspace(voltToDbm(val, 0), voltToDbm(val+val*read_samples*ramp_rate, 0), read_samples)
#     wv_p2 = np.linspace(voltToDbm(-val, 0), voltToDbm(-val-val*read_samples*ramp_rate, 0), read_samples)

#     waves_p1.append(wv_p1)
#     waves_p2.append(wv_p2)


# Sequence

*P1/P2 sequence explanation*: The sequence starts at a "home" point with 0 V on the P1/P2/J pulsing lines. After waiting for a trigger, the sequence will immediately go to it's initialisation point (executeTableEntry(0)) and wait for a set period of time (here: wait_and_settle, read_len). After a J pulse between wait_and_settle and read_len events P1 and P2 will be pulsed to an arbitrary point (executeTableEntry(1)) which is uploaded and updated every time in python after SET feedback has occured.

In [ ]:
seqc_program_p1 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_p2 = f"""
// Assign a single channel waveform to wave table entry 0

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    executeTableEntry(0);                                                               // mixed state init.
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read reference

    executeTableEntry(1);                                                               // read
    
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""

seqc_program_j = f"""
// Assign a single channel waveform to wave table entry 0
wave w_j = ones({timeToSamples(init_len, samplingDivider)});
assignWaveIndex(1,2, w_j, 0);

// Reset the oscillator phase
resetOscPhase();

repeat({seq_averages}) {{

    // Trigger the scope
    
    waitDigTrigger(1);

    setTrigger(1);
    setTrigger(0);
    
    playZero({timeToSamples(init_len, samplingDivider)},  {samplingDivider});           // mixed state init.
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle

    executeTableEntry(1);                                                               // read reference
    
    playZero({timeToSamples(read_len, samplingDivider)},  {samplingDivider});           // read
    playZero({timeToSamples(wait_and_settle, samplingDivider)},  {samplingDivider});    // wait and settle
}}
"""